In [1]:
from datasets import Dataset, load_dataset
import pandas as pd
from PIL import Image
import numpy as np
from io import BytesIO

/Users/arnavakula/.pyenv/versions/3.12.4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("A2H0H0R1/plant-disease-new", split="train")

In [3]:
len(dataset)

162916

In [4]:
arr = np.array(dataset[0]['image'])

In [5]:
type(Image.fromarray(arr))

PIL.Image.Image

In [6]:
data = []

for i in range(len(dataset) // 20):
    img = np.array(dataset[i]['image'])
    label = dataset[i]['label']
    data.append({'image': img, 'label': label})

In [7]:
df = pd.DataFrame(data)

In [8]:
df.head()

,image,label
0,"[[[167, 163, 196], [166, 162, 195], [165, 161,...",0
1,"[[[135, 128, 179], [130, 123, 174], [126, 119,...",0
2,"[[[123, 135, 177], [124, 136, 178], [124, 136,...",0
3,"[[[234, 225, 242], [191, 182, 199], [183, 174,...",0
4,"[[[143, 139, 172], [145, 141, 174], [147, 143,...",0


In [9]:
df.shape

(8145, 2)

In [10]:
len(df['label'].unique())

4

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [12]:
images = np.stack(df['image'].values)
labels = df['label'].values

In [13]:
images = images / 255.0
labels = to_categorical(labels, num_classes=12)

In [27]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(12, activation='softmax')
])

In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
history = model.fit(images, labels)

255/255 ━━━━━━━━━━━━━━━━━━━━ 165s 642ms/step - accuracy: 0.6269 - loss: 1.0857


In [35]:
model.save('../cnn-models/model.keras')